In [1]:
import os
import pandas as pd
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
location='../data/train'
classes=[]
videos=[]
train=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
train['video_name']=videos
print(classes)
train.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
1,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
2,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
3,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
4,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...


In [3]:
location='../data/test'
classes=[]
videos=[]
test=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
test['video_name']=videos
print(classes)
test.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
1,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
2,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
3,brush_hair/Blonde_being_brushed_brush_hair_f_n...
4,brush_hair/Blonde_being_brushed_brush_hair_u_c...


In [4]:
location='../data/val'
classes=[]
videos=[]
val=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
val['video_name']=videos
print(classes)
val.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UP...
1,brush_hair/Brushing_hair__the_right_way_brush_...
2,brush_hair/Brushing_my_Long_Hair__February_200...
3,brush_hair/Ella_brushing_her_amazing_long_hair...
4,brush_hair/Haarek_mmen_brush_hair_h_cm_np1_fr_...


In [5]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

In [6]:
# creating tags for test videos
val_video_tag = []
for i in range(val.shape[0]):
    val_video_tag.append(val['video_name'][i].split('/')[0])
    
val['tag'] = val_video_tag

In [7]:
# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [9]:
train_data = pd.DataFrame(columns=['image','class'])
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    classname=videoFile.split(' ')[0].split('/')[0]
#    print('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])
    cap = cv2.VideoCapture('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    frameRate=frameRate/2
    ret, frame1 = cap.read()
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#    print('length')
#    print(length)
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    x=1
    while(x<length):
#        print(x)
        frameId = cap.get(1) #current frame number
        ret, frame2 = cap.read()
        x+=1
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            name=(videoFile.split('/')[1]).split('.')[0];
            filename ='../data/train_frame_OF/' + name +"_flow%d.jpg" % count;
            imgname=name +"_flow%d.jpg" % count  
            count+=1          
            train_data=train_data.append({'image':imgname,'class':classname},ignore_index=True)
#            print(filename)
            exist=os.path.exists(filename)
            if exist==False:
                # storing the frames in a new folder named train_1
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                hsv[...,0] = ang*180/np.pi/2
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                cv2.imwrite(filename, rgb)
#            print(filename)
#            print(imgname)         
            train_data=train_data.append({'image':imgname,'class':classname},ignore_index=True)
    cap.release()


  9%|▉         | 325/3570 [02:29<18:45,  2.88it/s]


 18%|█▊        | 649/3570 [04:36<18:49,  2.59it/s]


 30%|██▉       | 1065/3570 [05:46<04:46,  8.75it/s]


 44%|████▍     | 1564/3570 [06:36<04:07,  8.11it/s]


 53%|█████▎    | 1890/3570 [07:48<03:29,  8.01it/s]


 63%|██████▎   | 2237/3570 [08:45<05:30,  4.03it/s]


 72%|███████▏  | 2579/3570 [09:55<03:44,  4.42it/s]


 82%|████████▏ | 2916/3570 [11:03<01:35,  6.83it/s]


 91%|█████████▏| 3261/3570 [12:18<01:25,  3.63it/s]


100%|██████████| 3570/3570 [13:26<00:00,  4.81it/s]

In [10]:
train_data.head()

,image,class
0,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
1,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
2,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
3,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
4,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair


In [11]:
train_data.nunique()

image    21189
class       51
dtype: int64

In [12]:
# converting the dataframe into csv file 
train_data.to_csv('../data/train_OF.csv',header=True, index=False)

In [13]:
train = pd.read_csv('../data/train_OF.csv')
train.head()

,image,class
0,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
1,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
2,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
3,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
4,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair


In [14]:
val_data = pd.DataFrame(columns=['image','class'])
# storing the frames from training videos
for i in tqdm(range(val.shape[0])):
    count = 0
    videoFile = val['video_name'][i]
    classname=videoFile.split(' ')[0].split('/')[0]
#    print('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])
    cap = cv2.VideoCapture('../data/val/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    frameRate=frameRate/2
    ret, frame1 = cap.read()
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#    print('length')
#    print(length)
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    x=1
    while(x<length):
#        print(x)
        frameId = cap.get(1) #current frame number
        ret, frame2 = cap.read()
        x+=1
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            name=(videoFile.split('/')[1]).split('.')[0];
            filename ='../data/val_frame_OF/' + name +"_flow%d.jpg" % count;
            imgname=name +"_flow%d.jpg" % count   
            count+=1         
            train_data=train_data.append({'image':imgname,'class':classname},ignore_index=True)
#            print(filename)
            exist=os.path.exists(filename)
            if exist==False:
                # storing the frames in a new folder named train_1
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                hsv[...,0] = ang*180/np.pi/2
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                cv2.imwrite(filename, rgb)
#            print(filename)
#            print(imgname)         
            val_data=val_data.append({'image':imgname,'class':classname},ignore_index=True)
    cap.release()


  0%|          | 0/1666 [00:00<?, ?it/s]

../data/val_frame_OF/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow0.jpg
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow0.jpg
../data/val_frame_OF/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow1.jpg
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow1.jpg
../data/val_frame_OF/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow2.jpg
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow2.jpg
../data/val_frame_OF/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow3.jpg
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow3.jpg
../data/val_frame_OF/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow4.jpg
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3_flow4.jpg
../data/val_frame_OF/Brushing_Her_Hair__[_NEW_AUDIO_]_U


  0%|          | 1/1666 [00:00<07:11,  3.86it/s]

../data/val_frame_OF/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow0.jpg
Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow0.jpg
../data/val_frame_OF/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow1.jpg
Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow1.jpg
../data/val_frame_OF/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow2.jpg
Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow2.jpg
../data/val_frame_OF/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow3.jpg
Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow3.jpg
../data/val_frame_OF/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow4.jpg
Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow4.jpg
../data/val_frame_OF/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow5.jpg
Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2_flow5.jpg
../data/val_frame_OF/Brushing_hair__the_right_way_brush_ha


  0%|          | 2/1666 [00:01<14:28,  1.92it/s]

...


100%|█████████▉| 1664/1666 [05:09<00:00,  5.90it/s]

../data/val_frame_OF/Glory_wave_u_nm_np6_fr_med_33_flow0.jpg
Glory_wave_u_nm_np6_fr_med_33_flow0.jpg
../data/val_frame_OF/Glory_wave_u_nm_np6_fr_med_33_flow1.jpg
Glory_wave_u_nm_np6_fr_med_33_flow1.jpg
../data/val_frame_OF/Glory_wave_u_nm_np6_fr_med_33_flow2.jpg
Glory_wave_u_nm_np6_fr_med_33_flow2.jpg
../data/val_frame_OF/Glory_wave_u_nm_np6_fr_med_33_flow3.jpg
Glory_wave_u_nm_np6_fr_med_33_flow3.jpg



100%|█████████▉| 1665/1666 [05:09<00:00,  6.22it/s]

../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow0.jpg
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow0.jpg
../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow1.jpg
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow1.jpg
../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow2.jpg
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow2.jpg
../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow3.jpg
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow3.jpg
../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow4.jpg
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow4.jpg
../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow5.jpg
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow5.jpg
../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow6.jpg
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3_flow6.jpg
../data/val_frame_OF/prelinger_LetsBeGo1953_wave_u_cm_n


100%|██████████| 1666/1666 [05:09<00:00,  4.39it/s]

In [15]:
val_data.nunique()

image    8803
class      51
dtype: int64

In [16]:
# converting the dataframe into csv file 
val_data.to_csv('../data/val_OF.csv',header=True, index=False)

In [17]:
val = pd.read_csv('../data/val_OF.csv')
val.head()

,image,class
0,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
1,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
2,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
3,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
4,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
